### 设置与组件定义 (Setup & Component Definitions)

说明: 这是我们的准备工作区。我们导入所有必要的库，定义我们将要在示例中使用的所有 AnalysisComponent，并准备好初始数据。为了演示清晰，我在每个组件的 run 方法中都加入了一条 print 语句，这样在执行时可以清楚地看到哪个组件正在运行。

In [ ]:
# Cell 1: Setup, Imports, and Component Definitions
import pandas as pd
import numpy as np
from typing import List, Dict

# Core imports for the new component-based system
from axisfuzzy.analysis._pipeline import FuzzyPipeline
from axisfuzzy.analysis._components.base import AnalysisComponent
from axisfuzzy.analysis.contracts import contract
from axisfuzzy.analysis.dataframe import FuzzyDataFrame
from axisfuzzy.fuzzifier import Fuzzifier

# Ensure the accessor is registered
import axisfuzzy.analysis.accessor

# --- Define a suite of mock _components for our examples ---

class FuzzifyComponent(AnalysisComponent):
    """Converts a crisp DataFrame to a FuzzyDataFrame."""
    def __init__(self, fuzzifier: Fuzzifier):
        self.fuzzifier = fuzzifier

    @contract(inputs={'data': 'CrispTable'}, outputs={'result': 'FuzzyTable'})
    def run(self, data: pd.DataFrame) -> FuzzyDataFrame:
        print("--- Executing: FuzzifyComponent.run ---")
        return FuzzyDataFrame.from_pandas(data, self.fuzzifier)

class UniformWeightComponent(AnalysisComponent):
    """Calculates uniform weights based on the number of columns."""
    @contract(inputs={'matrix': 'FuzzyTable'}, outputs={'weights': 'WeightVector'})
    def run(self, matrix: FuzzyDataFrame) -> np.ndarray:
        print("--- Executing: UniformWeightComponent.run ---")
        return np.full(matrix.shape[1], 1 / matrix.shape[1])

class ConstantWeightComponent(AnalysisComponent):
    """Returns a predefined constant weight vector."""
    def __init__(self, const_weights: list):
        self.const_weights = np.array(const_weights)

    @contract(inputs={'matrix': 'FuzzyTable'}, outputs={'weights': 'WeightVector'})
    def run(self, matrix: FuzzyDataFrame) -> np.ndarray:
        # matrix is an input to ensure it fits in the DAG, even if unused.
        print(f"--- Executing: ConstantWeightComponent.run with weights {self.const_weights} ---")
        return self.const_weights

class AggregateComponent(AnalysisComponent):
    """A mock aggregation component."""
    @contract(inputs={"matrix": "FuzzyTable", "weights": "WeightVector"}, outputs={"scores": "ScoreVector"})
    def run(self, matrix: FuzzyDataFrame, weights: np.ndarray) -> np.ndarray:
        print(f"--- Executing: AggregateComponent.run ---")
        # Simplified aggregation: defuzzify with a basic score and then aggregate
        crisp_matrix = np.zeros(matrix.shape, dtype=float)
        for j, col_name in enumerate(matrix.columns):
            for i in range(len(matrix)):
                 # Using a simple score for demonstration
                crisp_matrix[i, j] = (matrix[col_name][i].md - matrix[col_name][i].nmd + 1) / 2
        return np.dot(crisp_matrix, weights)

class DecisionComponent(AnalysisComponent):
    """A mock multi-output decision component."""
    @contract(inputs={"scores": "ScoreVector", "alternative_names": "RankingResult"}, outputs={"ranking": "RankingResult", "top_performer": "str"})
    def run(self, scores: np.ndarray, alternative_names: list) -> dict:
        print(f"--- Executing: DecisionComponent.run ---")
        ranking_indices = np.argsort(scores)[::-1]  # Sort descending
        return {
            "ranking": [alternative_names[i] for i in ranking_indices],
            "top_performer": alternative_names[ranking_indices[0]]
        }

# --- Prepare initial data and component instances ---
crisp_df = pd.DataFrame({
    'cost': [0.5, 0.7, 0.3],
    'safety': [0.2, 0.9, 0.4],
    'comfort': [0.8, 0.3, 0.1]
}, index=['CarA', 'CarB', 'CarC'])

fuzzifier_engine = Fuzzifier(mtype='qrofn', mf='GaussianMF', mf_params=[{"sigma": 0.25, "c": 0.5}])

print("✅ Setup complete. All _components are defined and data is ready.")

### 示例 1 - 简单的线性管道 (通过 Accessor 执行)

说明: 这是最基础的用例，展示了从一个 pandas.DataFrame 开始，通过 .fuzzy 访问器运行一个线性工作流，并直接获取最终结果。

In [ ]:
# Cell 2: Example 1 - Simple Linear Pipeline via Accessor
print("### Example 1: Simple Linear Pipeline ###\n")

# 1. Instantiate the required _components
fuzzifier = FuzzifyComponent(fuzzifier=fuzzifier_engine)
weighter = UniformWeightComponent()

# 2. Define the pipeline
p_linear = FuzzyPipeline()

# Input name can be anything because there's only one input for the accessor
crisp_input = p_linear.input("my_data", contract="CrispTable")
fuzz_table = p_linear.add(fuzzifier.run, data=crisp_input)
weights = p_linear.add(weighter.run, matrix=fuzz_table)

print("Pipeline defined:")
print(p_linear)
print("-" * 20)

# 3. Execute using the FuzzyAccessor
# The `crisp_df` is automatically injected into the single input node.
final_weights = crisp_df.fuzzy.run(p_linear)

# 4. Check the result
print("\n--- Execution Finished ---")
print(f"Type of final result: {type(final_weights)}")
print(f"Final weights: {final_weights}")

# Because the pipeline has a single, unambiguous terminal node, the result is returned directly.
assert isinstance(final_weights, np.ndarray)

### 示例 2 - 复杂的非线性 DAG (多源输入与多路输出)

说明: 这个例子展示了我们框架处理分支和合并的能力，以及如何处理具有多个最终输出的管道。

In [ ]:
# Cell 3: Example 2 - Non-linear DAG with Multiple Outputs
print("### Example 2: Non-linear DAG ###\n")

# 1. Instantiate _components
fuzzifier = FuzzifyComponent(fuzzifier=fuzzifier_engine)
const_weighter = ConstantWeightComponent(const_weights=[0.6, 0.1, 0.3])
aggregator = AggregateComponent()
decider = DecisionComponent()

# 2. Define the pipeline
p_dag = FuzzyPipeline()

# Define two inputs for the graph
crisp_input = p_dag.input("init_data", contract="CrispTable")
alt_names_input = p_dag.input("alternative_names", contract="RankingResult")

# Branch A: Fuzzification
fuzz_table = p_dag.add(fuzzifier.run, data=crisp_input)

# Branch B: Get some constant weights (this also depends on fuzz_table in our mock component)
const_weights = p_dag.add(const_weighter.run, matrix=fuzz_table)

# Merge branches: Use the results from both branches as input for the next step
scores = p_dag.add(aggregator.run, matrix=fuzz_table, weights=const_weights)

# Final step with multiple outputs
# This is now the single terminal node of the graph.
decision_outputs = p_dag.add(decider.run, scores=scores, alternative_names=alt_names_input)

print("Pipeline defined:")
print(p_dag)
print("-" * 20)

# 3. Execute the pipeline
# The terminal node `decider.run` returns a dictionary, so the final result will be that dictionary.
final_decision = p_dag.run({
    "init_data": crisp_df,
    "alternative_names": list(crisp_df.index)
})

# 4. Check the result
print("\n--- Execution Finished ---")
print(f"Type of final result: {type(final_decision)}")
print("Final decision results:")
import json
print(json.dumps(final_decision, indent=2))

assert isinstance(final_decision, dict)
assert "ranking" in final_decision
assert final_decision['top_performer'] == 'CarA'

### 示例 3 - 直接从 FuzzyDataFrame 开始

说明: 这个例子展示了框架的灵活性，允许用户跳过模糊化步骤，直接从已有的模糊数据开始分析。

In [ ]:
# Cell 4: Example 3 - Starting with a FuzzyDataFrame
print("### Example 3: Starting with a FuzzyDataFrame ###\n")

# 1. Create a pre-fuzzified DataFrame
fuzzy_df = FuzzifyComponent(fuzzifier=fuzzifier_engine).run(crisp_df)
print("Created a FuzzyDataFrame to start with:")
print(fuzzy_df)
print("-" * 20)

# 2. Instantiate the required component
weighter = UniformWeightComponent()

# 3. Define a pipeline that expects a FuzzyTable
p_from_fuzzy = FuzzyPipeline()
fuzzy_input = p_from_fuzzy.input(contract="FuzzyTable") # Name is omitted for single input
weights = p_from_fuzzy.add(weighter.run, matrix=fuzzy_input)

print("Pipeline defined:")
print(p_from_fuzzy)
print("-" * 20)

# 4. Execute by calling p.run() directly and passing the FuzzyDataFrame
# We don't use the accessor here, as we are not starting from a crisp df.
final_weights = p_from_fuzzy.run(fuzzy_df)

# 5. Check the result
print("\n--- Execution Finished ---")
print(f"Type of final result: {type(final_weights)}")
print(f"Final weights: {final_weights}")

assert np.allclose(final_weights, [1/3, 1/3, 1/3])

### 示例 4 - 返回中间结果

说明: 这个例子展示了如何使用 return_intermediate=True 来调试和检查管道中每一步的输出。

In [ ]:
# Cell 5: Example 4 - Returning Intermediate Results
print("### Example 4: Returning Intermediate Results ###\n")

# We reuse the linear pipeline from Example 1
fuzzifier = FuzzifyComponent(fuzzifier=fuzzifier_engine)
weighter = UniformWeightComponent()

p_linear = FuzzyPipeline()

crisp_input = p_linear.input(contract="CrispTable")
fuzz_table = p_linear.add(fuzzifier.run, data=crisp_input)
weights = p_linear.add(weighter.run, matrix=fuzz_table)

# Execute with return_intermediate=True
final_output, intermediate_states = crisp_df.fuzzy.run(p_linear, return_intermediate=True)

print("--- Execution Finished ---\n")

print("Final Output:")
print(final_output)
print("\n" + "="*40 + "\n")

print("Intermediate States (a dictionary mapping step_id to its result):")
# We can iterate through the states to see what each step produced
for step_id, result in intermediate_states.items():
    # Find the display name for this step_id for better readability
    step_info = p_linear.get_step_info(step_id)
    display_name = step_info.display_name

    print(f"\n--- State after step '{display_name}' (id: {step_id[:8]}) ---")
    # Pretty print DataFrames
    if isinstance(result, (pd.DataFrame, FuzzyDataFrame)):
        print(result)
    else:
        print(np.round(result, 4)) # Round arrays for cleaner output